In [1]:
import json
import os
import pandas as pd
from datetime import datetime, timedelta
from time import sleep
from itertools import combinations
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By

from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
import warnings
warnings.filterwarnings("ignore")

In [2]:
def init_driver(url):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH,options=chrome_options)
    driver.maximize_window()
    try:
        driver.get(url)
    except:
        print('error 4')
            
    return driver


def wait_for_page_load(xpath_url):
    global driver
    try:
        element_present = EC.presence_of_element_located((By.XPATH, xpath_url))
        WebDriverWait(driver, 5).until(element_present)
        return 1
    except TimeoutException:
        
        print("Timed out waiting for page to load")
        try:
            all_text = driver.find_element_by_xpath('//div[@class="_2jvV"]').text
            if 'No flights found' in all_text:
                return 2
            
            driver.close()
        except:
            driver = init_driver(url)
            print('driver not found')
            # sleep(2)
        
        return False
    except NoSuchElementException:
        print("Element not found")
        return False

In [9]:
print('Starting...')
url = 'https://paytm.com/flights'
CHROMEDRIVER_PATH = r"E:\New folder\chromedriver.exe"
WINDOW_SIZE = "1920,1080"
chrome_options = Options()
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_experimental_option("useAutomationExtension", False)
chrome_options.add_experimental_option("excludeSwitches",["enable-automation"])
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

print('initiated...')

preferences = {
	# "profile.default_content_settings.popups": 0,
	"download.default_directory": os.getcwd() + os.path.sep,
	# "directory_upgrade": True
}
chrome_options.add_experimental_option('prefs', preferences)

# chrome_options.binary_location = CHROME_PATH

driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH, options=chrome_options)
driver.maximize_window()


driver.get(url)

Starting...
initiated...


In [10]:
pd.options.display.max_rows = 100

In [11]:
citys = [
    'DEL-Delhi',
    'BOM-Mumbai',
    'BLR-Bengaluru',
    'HYD-Hyderabad',
    'CU-Kolkata',
    'MAA-Chennai',
    'AMD-Ahmedabad',
    'GOI-Goa',
    'PAT-Patna',
    'COK-Kochi',
    'LKO-Lucknow',
    'GAU-Guwahati',
    'PNQ-Pune',
    'JAI-Jaipur',
    'SXR-Srinagar',
    'BBI-Bhubaneshwar',
    'IXB-Bagdogra',
    'VNS-Varanasi',
    'IXC-Chandigarh',
    'IXR-Ranchi',
    'VTZ-Vishakhapatnam',
    'RPR-Raipur',
    'NAG-Nagpur',
    'TRV-Thiruvananthapuram',
    'CCJ-Kozhikode',
    'IDR-Indore',
    'ATQ-Amritsar',
    'IXJ-Jammu',
    'CJB-Coimbatore',
    'DED-Dehradun',
]

In [12]:
possible_combinations = list(combinations(citys,2))

In [14]:
def collect_data(driver, today, next_date, day_before, source, destination):
    
    status_1 = wait_for_page_load('//div[@class="_2JLq"]/div[@class="_2TFk"]')
    status_2 = wait_for_page_load('//div[@class="_3H-S _1Eia"]')
    
    if status_1 == 2 and status_2 == 2:
        print('flight not found')
        return
    
    all_boxes = driver.find_elements_by_xpath('//div[@class="_2JLq"]/div[@class="_2TFk"]')
    try:
        for one_box in all_boxes:
            try:
                flight_name = one_box.find_element_by_xpath('.//div[@class="_3H-S _1Eia"]').text
                flight_id = one_box.find_element_by_xpath('.//div[@class="NqXj _2GoO"]').text
                start_time = one_box.find_element_by_xpath('.//div[@class="_3H-S"]').text
                end_time = one_box.find_element_by_xpath('.//div[@class="_3H-S _1wD5"]').text.split('\n')[0]
                travel_time = one_box.find_element_by_xpath('.//div[@class="vY4t"]').text
                stops = one_box.find_element_by_xpath('.//div[@class="_7BOG"]').text
                fare = one_box.find_element_by_xpath('.//div[@class="_1cxG"]').text
                 
                all_data.append({
                    'flight_name': flight_name,
                    'flight_id': flight_id,
                    'start_time': start_time,
                    'end_time': end_time,
                    'travel_time': travel_time,
                    'stops': stops,
                    'fare': fare,
                    'source': source,
                    'destination': destination
                })
            except Exception as e:
                print(e)
                print('error2')
                # sleep(5)
                
    except Exception as e:
        print(e)
        print('error3')
        # sleep(5)

In [15]:
def search_page(driver, source, destination):
    
    try:
        clear_inputs = driver.find_elements_by_xpath('//div[@class="fl-input-clear"]')
        clear_inputs[0].click()
        clear_inputs[1].click()
    except:
        print('cleared already')
        
    sleep(2)
    wait_for_page_load('//div[@class="_31PN"]//input')
    
    inputs = driver.find_elements_by_xpath('//div[@class="_31PN"]//input')

    start_input = inputs[0]
    end_input = inputs[1]

    start_input.clear()
    start_input.send_keys(source)
    sleep(2)
    wait_for_page_load('//div[@class="_2Alu"]//div[@class="_2xgL"]')
    # Select First suggestion
    driver.find_element_by_xpath('//div[@class="_2Alu"]//div[@class="_2xgL"]').click()

    end_input.clear()
    end_input.send_keys(destination)
    sleep(2)
    
    wait_for_page_load('//div[@class="_2Alu"]//div[@class="_2xgL"]')
    # Select First suggestion
    driver.find_element_by_xpath('//div[@class="_2Alu"]//div[@class="_2xgL"]').click()

    driver.find_element_by_xpath("//button[text()='Search']").click()
    sleep(2)

In [16]:
all_data = []
for i, one_combination in enumerate(possible_combinations):
    source, destination = one_combination
    today = datetime.today() + timedelta(5)
    print(today)
    try:
        next_date = today.strftime('%Y-%m-%d') # (today + timedelta(i)).strftime('%Y-%m-%d')
        print(next_date)
        search_page(driver, source, destination)
        sleep(2)
        collect_data(driver, today, next_date, 0, source, destination)
    except Exception as e:
        # sleep(2)
        print(e)
        print('error1')
        try:
            driver.close()
        except:
            print('driver not found')
            pd.DataFrame(all_data).to_csv('flight_data__.csv')
            
        driver = init_driver(url)
        
    print(one_combination, i+1, len(possible_combinations), (i+1)/ len(possible_combinations))
df = pd.DataFrame(all_data)
df

2022-10-24 11:37:29.697680
2022-10-24
cleared already
('DEL-Delhi', 'BOM-Mumbai') 1 435 0.0022988505747126436
2022-10-24 11:38:06.717111
2022-10-24
('DEL-Delhi', 'BLR-Bengaluru') 2 435 0.004597701149425287
2022-10-24 11:38:40.088958
2022-10-24
('DEL-Delhi', 'HYD-Hyderabad') 3 435 0.006896551724137931
2022-10-24 11:38:59.907921
2022-10-24
('DEL-Delhi', 'CU-Kolkata') 4 435 0.009195402298850575
2022-10-24 11:39:25.230394
2022-10-24
('DEL-Delhi', 'MAA-Chennai') 5 435 0.011494252873563218
2022-10-24 11:39:50.128250
2022-10-24
('DEL-Delhi', 'AMD-Ahmedabad') 6 435 0.013793103448275862
2022-10-24 11:40:15.839528
2022-10-24
('DEL-Delhi', 'GOI-Goa') 7 435 0.016091954022988506
2022-10-24 11:40:47.519909
2022-10-24
('DEL-Delhi', 'PAT-Patna') 8 435 0.01839080459770115
2022-10-24 11:41:02.408810
2022-10-24
('DEL-Delhi', 'COK-Kochi') 9 435 0.020689655172413793
2022-10-24 11:41:19.464503
2022-10-24
('DEL-Delhi', 'LKO-Lucknow') 10 435 0.022988505747126436
2022-10-24 11:41:33.900304
2022-10-24
('DEL-Del

,flight_name,flight_id,start_time,end_time,travel_time,stops,fare,source,destination
0,Go First,G8 - 713,19:15,00:20,5h 5m,1 stop at Ahmedabad,"6,592",DEL-Delhi,BOM-Mumbai
1,Go First,G8 - 713,19:15,09:05,13h 50m,1 stop at Ahmedabad,"6,906",DEL-Delhi,BOM-Mumbai
2,Go First,G8 - 336,15:15,17:20,2h 5m,Non Stop,"7,319",DEL-Delhi,BOM-Mumbai
3,Go First,G8 - 323,18:20,20:35,2h 15m,Non Stop,"7,319",DEL-Delhi,BOM-Mumbai
4,Go First,G8 - 330,20:50,23:05,2h 15m,Non Stop,"7,319",DEL-Delhi,BOM-Mumbai
...,...,...,...,...,...,...,...,...,...
4317,IndiGo,6E - 779,20:05,07:35,11h 30m,1 stop at Hyderabad,"13,713",IXJ-Jammu,DED-Dehradun
4318,,,,,,,,IXJ-Jammu,DED-Dehradun
4319,IndiGo,6E - 2189,21:40,06:45,9h 5m,1 stop at Delhi,"6,877",CJB-Coimbatore,DED-Dehradun
4320,IndiGo,6E - 779,20:05,07:35,11h 30m,1 stop at Hyderabad,"13,713",CJB-Coimbatore,DED-Dehradun


In [17]:
df.to_csv('flight_data_paytm.csv')